In [ ]:
import numpy as np
import pandas as pd
import ast
from nltk.stem.porter import PorterStemmer as PS
from sklearn.feature_extraction.text import CountVectorizer as CV
from sklearn.metrics.pairwise import cosine_similarity as cs
cv = CV(max_features=5000,stop_words='english')
ps=PS()

Import

In [ ]:
movies_dataset=pd.read_csv("Raw_Dataset\\tmdb_5000_movies.csv")
credits_dataset=pd.read_csv(r"Raw_Dataset\tmdb_5000_credits.csv")

In [ ]:
movies_dataset=movies_dataset.merge(credits_dataset,on='title')

In [ ]:
movies_dataset=movies_dataset[['movie_id','title','overview','genres','keywords','cast','crew']]
movies_dataset.dropna(inplace=True)
movies_dataset.head(1)


In [ ]:
movies_dataset.iloc[0].genres

In [ ]:
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
        
    return l
    

In [ ]:
movies_dataset['genres']=movies_dataset['genres'].apply(convert)

movies_dataset['keywords']=movies_dataset['keywords'].apply(convert)


In [ ]:
def convert_top(obj):
    l=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            l.append(i['name'])
            counter+=1
        else:
            break
        
    return l

In [ ]:
movies_dataset['cast']=movies_dataset['cast'].apply(convert_top)

In [ ]:
def fetch_director(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
    return l

In [ ]:
movies_dataset['crew']=movies_dataset['crew'].apply(fetch_director)

In [ ]:
movies_dataset['crew'].head()

In [ ]:
movies_dataset['overview']=movies_dataset['overview'].apply(lambda x:x.split())

In [ ]:
cols=['genres','keywords','cast','crew']

for col in cols:
    movies_dataset[col]=movies_dataset[col].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies_dataset.head(1)

In [ ]:
movies_dataset['tags']=movies_dataset['overview']+movies_dataset['genres']+movies_dataset['keywords']+movies_dataset['cast']+movies_dataset['crew']

In [ ]:
preprocessed_movies=movies_dataset[['movie_id','title','tags']]

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i)) 
    return " ".join(y)

In [ ]:
preprocessed_movies['tags'] = preprocessed_movies['tags'].apply(lambda x:" ".join(x))

In [ ]:
preprocessed_movies['tags'] = preprocessed_movies['tags'].apply(lambda x:x.lower())

In [ ]:
preprocessed_movies['tags'] = preprocessed_movies['tags'].apply(stem)

In [ ]:
tags_vector=cv.fit_transform(preprocessed_movies['tags']).toarray()

In [ ]:
tags_vector[0]

In [ ]:
similarity = cs(tags_vector)

In [ ]:
def recommend(movie):
    movie_index = preprocessed_movies[preprocessed_movies['title']==movie].index[0]
    distances = similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(preprocessed_movies.iloc[i[0]].title)

In [ ]:
recommend("Batman Begins")